# Testing sentence tokenizers on legal text examples

### 1. Define a nice german sentence and a paragraph

In [6]:
sentence = u"So kann auch das freie Belieben einer mitwirkenden Partei zur Bedingung eines Rechtsgeschäfts gemacht werden, sog. Potestativbedingung (Palandt-Heinrichs, BGB, Einf. v. § 158 Rdnr. 10; RGZ 104, 100; BGHZ 47, 391; BGH NJW-RR 1996, 1167)."

In [7]:
paragraph = u"Der Kläger hat schließlich auch nicht in ausreichend substantiierter Weise dargelegt, dass er vom Beklagten arglistig getäuscht worden wäre (§ 123 Abs. 1 Alt. 1 BGB). Welche „falschen Versprechungen“ dem Kläger bei Vertragsschluss gemacht sein sollen, erläutert der Kläger auch auf Bestreiten der Beklagtenpartei in der Klageschrift nicht näher. Auch aus der informatorischen Anhörung des Klägers haben sich keine Anhaltspunkte für eine Täuschung bei Vertragsabschluss ergeben."

### 2. Use standard nltk german sentence tokenizer

In [8]:
import nltk

In [9]:
nltk_sent = nltk.data.load('tokenizers/punkt/german.pickle')

In [10]:
%time nltk_sent.sentences_from_text(sentence)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 538 µs


[u'So kann auch das freie Belieben einer mitwirkenden Partei zur Bedingung eines Rechtsgesch\xe4fts gemacht werden, sog.',
 u'Potestativbedingung (Palandt-Heinrichs, BGB, Einf.',
 u'v. \xa7 158 Rdnr.',
 u'10; RGZ 104, 100; BGHZ 47, 391; BGH NJW-RR 1996, 1167).']

#### We get a sentence split into 4 fragments because the standard sentence tokenizer model is underspecified (trained on a too general text genre)

In [6]:
%time nltk_sent.sentences_from_text(paragraph)

CPU times: user 2.28 ms, sys: 0 ns, total: 2.28 ms
Wall time: 630 µs


[u'Der Kl\xe4ger hat schlie\xdflich auch nicht in ausreichend substantiierter Weise dargelegt, dass er vom Beklagten arglistig get\xe4uscht worden w\xe4re (\xa7 123 Abs. 1 Alt.',
 u'1 BGB).',
 u'Welche \u201efalschen Versprechungen\u201c dem Kl\xe4ger bei Vertragsschluss gemacht sein sollen, erl\xe4utert der Kl\xe4ger auch auf Bestreiten der Beklagtenpartei in der Klageschrift nicht n\xe4her.',
 u'Auch aus der informatorischen Anh\xf6rung des Kl\xe4gers haben sich keine Anhaltspunkte f\xfcr eine T\xe4uschung bei Vertragsabschluss ergeben.']

### 3. Testing a retrained version of the nltk sentence tokenizer (model based on 1500 decision documents)

In [11]:
import hickle
jur_sent = hickle.load('../data/jursentok.hkl', safe=False)

In [12]:
%time jur_sent.sentences_from_text(sentence)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 770 µs


[u'So kann auch das freie Belieben einer mitwirkenden Partei zur Bedingung eines Rechtsgesch\xe4fts gemacht werden, sog. Potestativbedingung (Palandt-Heinrichs, BGB, Einf. v.',
 u'\xa7 158 Rdnr. 10; RGZ 104, 100; BGHZ 47, 391; BGH NJW-RR 1996, 1167).']

In [13]:
%time jur_sent.sentences_from_text(paragraph)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.1 ms


[u'Der Kl\xe4ger hat schlie\xdflich auch nicht in ausreichend substantiierter Weise dargelegt, dass er vom Beklagten arglistig get\xe4uscht worden w\xe4re (\xa7 123 Abs. 1 Alt.',
 u'1 BGB).',
 u'Welche \u201efalschen Versprechungen\u201c dem Kl\xe4ger bei Vertragsschluss gemacht sein sollen, erl\xe4utert der Kl\xe4ger auch auf Bestreiten der Beklagtenpartei in der Klageschrift nicht n\xe4her.',
 u'Auch aus der informatorischen Anh\xf6rung des Kl\xe4gers haben sich keine Anhaltspunkte f\xfcr eine T\xe4uschung bei Vertragsabschluss ergeben.']

#### Still errors are made, but false positives are reduced (look up mini-eval in own gscl paper draft)

### 4. Using an improved tokenizer that corrects false positives based on a legal abbreviation list after prob. model has been applied ( a little heuristic though)

In [14]:
from jursegtok import tokenizer

ImportError: cannot import name JurSentTokenizer

In [11]:
jur_sent_impr = tokenizer.JurSentTokenizer()

In [12]:
%time jur_sent_impr.sentence_tokenize(paragraph)

CPU times: user 742 µs, sys: 527 µs, total: 1.27 ms
Wall time: 1.22 ms


[u'Der Kl\xe4ger hat schlie\xdflich auch nicht in ausreichend substantiierter Weise dargelegt, dass er vom Beklagten arglistig get\xe4uscht worden w\xe4re (\xa7 123 Abs. 1 Alt. 1 BGB).',
 u'Welche \u201efalschen Versprechungen\u201c dem Kl\xe4ger bei Vertragsschluss gemacht sein sollen, erl\xe4utert der Kl\xe4ger auch auf Bestreiten der Beklagtenpartei in der Klageschrift nicht n\xe4her.',
 u'Auch aus der informatorischen Anh\xf6rung des Kl\xe4gers haben sich keine Anhaltspunkte f\xfcr eine T\xe4uschung bei Vertragsabschluss ergeben.']

In [16]:
%time jur_sent_impr.sentence_tokenize(sentence)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 860 µs


[u'So kann auch das freie Belieben einer mitwirkenden Partei zur Bedingung eines Rechtsgesch\xe4fts gemacht werden, sog. Potestativbedingung (Palandt-Heinrichs, BGB, Einf. v. \xa7 158 Rdnr. 10; RGZ 104, 100; BGHZ 47, 391; BGH NJW-RR 1996, 1167).']

### Errors are corrected but correction lookup heuristic is costly and performs slower